# Multiple Linear regression without Scikit Learn

There's nothing better to understand the gradient descent algorithm than to code it from scratch. What? you have heard this before ? This time we are switching to gradient descent for multiple linear regression!

Don't hesitate to come back to your Machine Learning course on linear regression to refresh your memory. 

Our goal will be to code a multiple linear regression such as : 

$f(x) = \beta \times x + \beta_0 = \beta_1 \times x_1 + \dots + \beta_p \times x_p + \beta_0$

* Import the following libraries: 
  * Numpy 

In [1]:
import numpy as np

* Define a `Model` class that will take two methods: 
  1. `__init__(self, data)`, where `data` will be the dataset containing the training variables. It's the class builder which will allow you to define an attribute $\beta_0$ (`beta_0` in your code) and an attribute $\beta$ (`beta` in your code). These attributes represent the coefficients/parameters of the model an we will be initialize them randomly using Numpy (cf: `np.random.randn`).
`beta` will have to contain a number of random values equal to the number of training variables.
  2. `__call__(self, x)`, a special method that will turn our class into a callable which will return $\beta \times x + \beta_0$ when called. 
  
  ⚠️ we are now working with matrices and vectors, therefore you will need to use operations that work for these objects ⚠️

In [57]:
class Model:

    def __init__(self, data):
        self.beta_0 = np.random.randn(1)
        self.beta = np.random.randn(data.shape[1])

    def __call__(self, x):
        return np.dot(self.beta, x.transpose()) + self.beta_0

* Import `sklearn.datasets`
  * Use the `load_diabetes()` function to load the diebetes dataset in an object called `diabetes`.
  * Print the `DESCR` attribute of the diabetes object
  * Save the content of the `data` attribute in an object named `diabetes_data`
  * Save the content of the `target` attribute in an object named `y`

In [52]:
from sklearn.datasets import load_diabetes

diabete = load_diabetes()
print(diabete.DESCR)

diabetes_data = diabete.data
y = diabete.target

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - Age
      - Sex
      - Body mass index
      - Average blood pressure
      - S1
      - S2
      - S3
      - S4
      - S5
      - S6

Note: Each of these 10 feature variables have been mean centered and scaled by the standard deviation times `n_samples` (i.e. the sum of squares of each column totals 1).

Source URL:
https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

For more information see:
Bra

* Create an instance of your class `Model` and display `beta_0` and `beta`

In [64]:
model = Model(diabetes_data)

In [59]:
# have a look at the attribute beta_0 (it's a random value)
model.beta_0

array([1.3797257])

In [60]:
# have a look at the attribute beta_1 (it's a random value)
model.beta

array([ 0.81195429, -0.91052717,  0.81345315,  0.5121951 ,  0.1102651 ,
       -0.87301279, -0.34923172,  0.76947408,  0.06407017, -2.48990095])

* Try doing a first "regression" by running `model(diabetes_data[0,:])`.   
NB: If you don't have the same values as this notebook in output, this is normal since you have initialized your values randomly. 

In [65]:
model(diabetes_data[0,:])

array([2.36585487])

* Visualize `y` against the predictions using `plotly`.

In [69]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter(x=y, y=model(diabetes_data))
fig.add_trace(go.Scatter(x=y, y=y, mode='lines'))
fig.show()

* Now we need to define a cost function. For a linear regression, we could use MSE : 

`np.mean((model(input) - y)**2)`

  * Create a function which we'll call `mse` (for mean square error). This function will take two arguments `y_pred` & `y_true`.

In [70]:
def mse(y_pred, y_true):
    return np.mean((y_pred - y_true)**2)

* Test your function by inserting `model(diabetes_data)` & `y` as arguments. 
* Calculate the rmse as well

In [71]:
y_pred = model(diabetes_data)
res_mse = mse(y_pred, y)
print(f"MSE : {res_mse}")
print(f"RMSE : {res_mse**0.5}")

MSE : 28414.30483786488
RMSE : 168.56543191848345


* We're going to need to compute the gradients for our variable `model.beta` and our constant `model.beta_0`. To do this, we're going to need to review our derivative formulas. Since we're not here to do math, we're going to give you these formulas. 
  * `derive_model_beta = 2/len(y_pred)*np.sum((x.transpose() @ (y_pred - y_true)))`
  * `derive_model_beta_0 = 2/len(y_pred)*(np.sum(y_pred - y_true))`

  * Feel free to read this article if you want to know more about the calculation of the derivative: [Gradient Descent Derivation](https://mccormickml.com/2014/03/04/gradient-descent-derivation/)


  * So using the above formulas, code the first function `derivative_mse_beta` that will take the arguments: 
    * `x` --> the values for your variable / `y_pred` --> the values predicted by your model / `y_true` --> the values of the target variable


In [75]:
# Calculate model.beta's derivative

def derivative_mse_beta(x, y_pred, y_true):
    return 2 / len(y_pred) * (x.transpose() @ (y_pred - y_true))

* Test you function

In [76]:
derivative_mse_beta(diabetes_data,y_pred, y)

array([-1.37209166, -0.3087123 , -4.30148336, -3.23437562, -1.55946983,
       -1.28151929,  2.89494962, -3.15917619, -4.14957215, -2.80571975])

* So using the above formulas, now code the `derivative_mse_beta_0` function which will take the arguments :
    * `y_pred` --> the values predicted by your model / `y_true` --> the actual values to predict

In [77]:
# Calculate model.beta_0's derivative

def derivative_mse_beta_0(y_pred, y_true):
    return 2/len(y_pred)*(np.sum(y_pred - y_true))

* Test you function

In [78]:
derivative_mse_beta_0(y_pred, y)

-299.85079420834705

* We will try to see if we can minimize our cost function using the two gradients above. To update our variables, we need to subtract their respective gradients. Ex: 
  * `param = param - learning_rate * gradient`

  * Set a `learning_rate` to 0.1
  * Try to apply your formula on `model.beta` and `model.beta_0`.

In [80]:
print(f"OLD model.beta_0 = {model.beta_0}")
print(f"OLD model.beta_1 = {model.beta}")
print(f"OLD model MSE = {mse(y_pred, y)}")

model.beta_0 -= 0.1 * derivative_mse_beta_0(y_pred, y)
model.beta -= 0.1 * derivative_mse_beta(diabetes_data, y_pred, y)
y_pred = model(diabetes_data)

print(f"NEW model.beta_0 = {model.beta_0}")
print(f"NEW model.beta_1 = {model.beta}")
print(f"NEW model MSE = {mse(y_pred, y)}")

OLD model.beta_0 = [32.19316648]
OLD model.beta_1 = [ 1.35429429  1.94611218 -0.65275372  0.01974768 -2.58513203  0.85235412
  1.53318071 -0.20287361  0.8292465  -0.69565526]
OLD model MSE = 28414.30483786488
NEW model.beta_0 = [62.1782459]
NEW model.beta_1 = [ 1.49150346  1.97698341 -0.22260538  0.34318524 -2.42918504  0.98050605
  1.24368574  0.11304401  1.24420371 -0.41508329]
NEW model MSE = 14020.784579928983


We see that the values of the two parameters have changed, let's see how it affected the predictions of the model. 
Visualize y vs the model's predictions.

In [81]:
fig = px.scatter(x=y, y=model(diabetes_data))
fig.add_trace(go.Scatter(x=y, y=y, mode='lines'))
fig.show()

We notice the predictions got a little closer to our real data

* Our MSE has dropped a lot! This is good news but the process of gradient descent is iterative. So you'll have to do it several times before arriving at accurate predictions. 
  * By making a loop, try to repeat the process from above 10,000 times. 
  * Display every 1000 epochs: mse, model.beta & model.beta_0 

In [ ]:
# Define learning rate and a number of iterations 

In [82]:
# Train your model
# Define learning rate and a number of iterations 

for i in range(0, 10000):
  y_pred = model(diabetes_data)
  model.beta_0 -= 0.1 * derivative_mse_beta_0(y_pred, y)
  model.beta -= 0.1 * derivative_mse_beta(diabetes_data, y_pred, y)
  loss = mse(y_pred, y)
  if not i % 1000:
    print(f"Epoch {i}".center(20, "-"))
    print(f"Current Loss: {loss}")
    print(f"beta_0 = {model.beta_0}")
    print(f"beta = {model.beta}")

------Epoch 0-------
Current Loss: 14020.784579928983
beta_0 = [80.16929355]
beta = [ 1.6284038   2.00761498  0.20700463  0.66616373 -2.27367026  1.1082264
  0.9546522   0.4283409   1.65855547 -0.13501528]
-----Epoch 1000-----
Current Loss: 3408.9746922722957
beta_0 = [152.13348416]
beta = [  49.17930282  -31.71018112  258.9196047   180.97206926   33.95934189
   11.87526191 -147.07163367  134.08600897  230.79532043  128.16435798]
-----Epoch 2000-----
Current Loss: 3080.027422429231
beta_0 = [152.13348416]
beta = [  38.72843512  -93.42453562  367.90748701  243.80714437    4.88224064
  -35.36524855 -185.09303346  143.63128105  313.13799458  143.24826237]
-----Epoch 3000-----
Current Loss: 2966.142808352611
beta_0 = [152.13348416]
beta = [  24.61903465 -141.67974856  427.90180125  274.34043054  -19.25494272
  -70.92909179 -198.19650518  137.76045206  357.98044219  137.09779054]
-----Epoch 4000-----
Current Loss: 2919.2638459193927
beta_0 = [152.13348416]
beta = [  13.96437756 -175.0907060

* Using `plotly`, view your model and actual values again

In [83]:
fig = px.scatter(x=y, y=model(diabetes_data))
fig.add_trace(go.Scatter(x=y, y=y, mode='lines'))
fig.show()

**We've got a nice regression this time!** 